In [1]:
import os
import random
import numpy as np
from tqdm import tqdm

import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import random
import copy
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
import shutil

%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.2)

HAPPY_COLORS_PALETTE = ["#01BEFE", "#FFDD00", "#FF7D00", "#FF006D", "#ADFF02", "#8F00FF"]

sns.set_palette(sns.color_palette(HAPPY_COLORS_PALETTE))

rcParams['figure.figsize'] = 12, 8

In [ ]:
npz_files = "../data/skel_out/hmdb51"

In [7]:
lol = np.load("../data/skel_out/hmbd51/brush_hair/April_09_brush_hair_u_nm_np1_ba_goo_2.npz")
coords, vid_size = lol["coords"],lol["video_size"]

In [8]:
coords.shape

(322, 33, 3)

### Add Skeleton of selected videos

In [9]:
#action_npz_file = "../data/skeleton_k10_v7_ movements.npz"
#action_arr = np.load(action_npz_file)

In [10]:
#for cls,arr,vid_n in zip(action_arr["arr_0"],action_arr["arr_1"],action_arr["arr_2"]):
#    os.makedirs(os.path.join(npz_files,f"{cls}"),exist_ok=True)
#    np.savez(os.path.join(npz_files,f"{cls}/{vid_n}.npz"),coords)

## Sequence Data

In [51]:
root_dir = "D:\\FYP\\HAR-ZSL-XAI\\AETraining"
main_dir = "D:\\FYP\\HAR-ZSL-XAI"
data_dir = os.path.join(main_dir,"data","skel_out","hmbd51")
refined_data = os.path.join(main_dir,"data","sequence_data","random_50f")
class_names = os.listdir(data_dir)
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 1-train_ratio - val_ratio
batch_size = 32

In [52]:
file_list = []

for class_n in os.listdir(data_dir):
    file_list += [os.path.join(data_dir,class_n,x) for x in os.listdir(os.path.join(data_dir,class_n))]

In [53]:
len(file_list)

8965

In [54]:
def break_into_time_frames(coords,vid_size,time_window=50,stride=40):
    shape = coords.shape
    [h,w] = vid_size

    #[x,y,z] is the format
    if shape[0]<time_window:
        return None

    frame_points = list(range(0,shape[0]-time_window,stride))
    arrays = []
    for i in frame_points:
        arrays.append(coords[i:i+time_window])
    return arrays

def random_break_into_time_frames(coords,vid_size,time_window=100,select=50,stride =40):
    shape = coords.shape
    [h,w] = vid_size

    #[x,y,z] is the format
    if shape[0]<time_window:
        return None

    frame_points = list(range(0,shape[0]-time_window,stride))
    arrays = []
    for i in frame_points:
        frame_no = sorted(random.sample(range(i,i+time_window),select))
        arrays.append(coords[frame_no])
    return arrays

def save_arrays(data_dir,file_name,class_name,arrays,vid_size):
    for __id,each_array in enumerate(arrays):
        np.savez(os.path.join(data_dir,f"{class_name}_cls_{file_name}_{__id}.npz"),coords=each_array,video_size=vid_size)


def classname_id(class_name_list):
    id2classname = {k:v for k, v in zip(list(range(len(class_name_list))),class_name_list)}
    classname2id = {v:k for k, v in id2classname.items()}
    return id2classname, classname2id

In [55]:
import cv2

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def gen_skeleton(frame, connections, height, width):
    img_3 = np.zeros([height, width,3],dtype=np.uint8)
    img_3.fill(255)

    # add circles
    for coord in frame:
        x, y = int(width*coord[0]), int(height*coord[1])
        img_3 = cv2.circle(img_3, center=(x,y), radius=1, color=(255, 0, 0), thickness=6)

    # add lines
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for line in mapping_list:
        i, j = line
        st = frame[i, :]
        start_point = (int(width*st[0]), int(height*st[1]))

        en = frame[j, :]
        end_point = (int(width*en[0]), int(height*en[1]))

        img3_ = cv2.line(img_3, start_point, end_point, color=(0, 0, 0), thickness=3)

    return img_3



def gen_video(points, save_file, frame_h, frame_w):
    # make 3D if points are flatten
    if len(points.shape) == 2:
        fts = points.shape[1]
        x_cds = list(range(0, fts, 3))
        y_cds = list(range(1, fts, 3))
        z_cds = list(range(2, fts, 3))
        points = np.transpose(np.array([points[:, x_cds], points[:, y_cds], points[:, z_cds]]), (1,2,0))

    size = (frame_w, frame_h)
    result = cv2.VideoWriter(save_file,
                         cv2.VideoWriter_fourcc(*'MJPG'),
                         10, size)

    # mapping_list = [(11, 12), (11, 13), (13, 15), (12, 14), (14, 16), (12, 24), (11, 23), (23, 24), (24, 26), (26, 28), (23, 25), (25, 27)]
    mapping_list = [(0, 1), (1, 3), (3, 5), (0, 2), (2, 4), (0, 6), (1, 7), (6, 7), (6, 8), (7, 9), (8, 10), (9, 11)]
    for __id,frame in enumerate(points):
        skel_image = gen_skeleton(frame, mapping_list, frame_h, frame_w)
        result.write(skel_image)

    result.release()

In [58]:
os.makedirs(refined_data,exist_ok=True)
for each_file in tqdm(file_list):
  a_file = np.load(each_file)
  coords, vid_size = a_file["coords"],a_file["video_size"]
  f_point = random_break_into_time_frames(coords,vid_size)

  path_parts = each_file.strip().split(os.path.sep)
  #print(path_parts)
  class_n = path_parts[-2]
  file_id = path_parts[-1].split(".")[0]

  #print(class_n, file_id)

  if f_point:
      gen_video(f_point[0], "check_vid.mp4", 400, 400)
      save_arrays(refined_data,file_id,class_n,f_point,vid_size)

100%|██████████| 8965/8965 [05:51<00:00, 25.48it/s] 


In [57]:
a_file = np.load("D:\\FYP\\HAR-ZSL-XAI\\data\\skel_out\\hmbd51\\run\\A_Beautiful_Mind_4_run_f_cm_np1_ba_med_9.npz")
coords, vid_size = a_file["coords"],a_file["video_size"]
f_point = random_break_into_time_frames(coords,vid_size)

path_parts = each_file.strip().split(os.path.sep)
#print(path_parts)
class_n = path_parts[-2]
file_id = path_parts[-1].split(".")[0]

#print(class_n, file_id)

if f_point:
    gen_video(a_file["coords"], "check_vid_main.mp4", 400, 400)
    gen_video(f_point[0], "check_vid.mp4", 400, 400)
    save_arrays(refined_data,file_id,class_n,f_point,vid_size)